In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class
feature_name = 'f__answer_removed'
df = df_item[~pd.isnull(df_item[feature_name])].copy()

In [ ]:
variables = list(df.variable_name.unique())

In [ ]:
from pyod.models.ecod import ECOD
model = ECOD(contamination=0.1)

df['anomaly'] = None
df['value'] = None
for index_range in range(0, len(variables)):
    mask = (df['variable_name'] == variables[index_range])
    #df.loc[mask,'anomaly'] = model.fit_predict(df[mask][[feature_name]])
    df.loc[mask,'value'] = df[mask][feature_name].apply(lambda x: 5 if x>5 else x)

In [ ]:
model = ECOD(contamination=0.15)
df['anomaly'] = model.fit_predict(df[['qnr_seq',feature_name]])

In [ ]:
print(df['anomaly'].value_counts(), df['anomaly'].value_counts()/df['anomaly'].count())

In [ ]:
mask = df['value']>0
for index_range in [x for x in range(0, len(variables), 50)]:
    plt.figure(figsize=(15, 6))
    sns.boxplot(df[(df['variable_name'].isin(variables[index_range:index_range+50]))&mask],x='variable_name', y='value', hue='anomaly')
    plt.xticks(rotation=90)
    plt.show()